In [159]:
from IPython.core.display import display, HTML, Markdown, Latex
from tabulate import tabulate
import pandas as pd

In [223]:
def num_percorsi(env):
    r=len(env)
    c=len(env[0])
    mat = [([0]*c) for _ in range(r)]
    for i in range(1,r):
        for j in range(1,c):
            if i==1 and j==1:
                mat[i][j]=1
            else:
                if mappa[i][j]=="*" or (mat[i][j-1]==None and mat[i-1][j]==None):
                    mat[i][j]=None
                else:
                    mat[i][j]=(mat[i-1][j] if mat[i-1][j] is not None else 0) + (mat[i][j-1] if mat[i][j-1] is not None else 0)
    return mat
   

#calcola il numero di percorsi possibili dalla cella (m,n) alla (1,1)

def num_percorsi_reverse(env):
    r=len(env)+1
    c=len(env[0])+1
    mat = [([0]*c) for _ in range(r)]
    for i in range(r-2,0,-1):
        for j in range(c-2,0,-1):
            if i==r-2 and j==c-2:
                mat[i][j]=1
            else:
                if mappa[i][j]=="*" or (mat[i+1][j]==None and mat[i][j+1]==None): 
                    mat[i][j]=None
                else:
                    mat[i][j]=(mat[i+1][j] if mat[i+1][j] is not None else 0) + (mat[i][j+1] if mat[i][j+1] is not None else 0)                          
    return mat

In [117]:
campo_minato = [
    [ " " , " " , " " , " " , " " , " " , " " , " " , "*" ],
    [ " " , " " , " " , " " , "*" , "*" , " " , " " , " " ],
    [ " " , "*" , " " , "*" , " " , " " , " " , " " , " " ],
    [ " " , " " , " " , " " , " " , " " , " " , "*" , " " ],
    [ " " , " " , "*" , " " , "*" , " " , " " , " " , " " ],
    [ " " , " " , " " , " " , " " , " " , "*" , " " , " " ],
    [ " " , "*" , " " , " " , " " , " " , " " , " " , " " ],
    [ " " , " " , " " , " " , " " , " " , " " , " " , " " ]
]
m = len(campo_minato)
n = len(campo_minato[0])
mappa = [ ["*"]*(n+1) ] + [ (["*"] + r) for r in campo_minato]

In [212]:
#def visualizza_mappa():
#    columns=[str(i) for i in range(1,n+1)]
#    index=pd.Index(["0"]+[chr(65+i) for i in range(m)])
#    df=pd.DataFrame(mappa,index=index)
#    df = df.iloc[1:,1:]
#    print(tabulate(df, headers=columns, tablefmt='fancy_grid'))
    
def visualizza_env(env):
    if len(env)==m+1 and len(env[0])==n+1:
        index=pd.Index(["0"]+[chr(65+i) for i in range(m)])
        df=pd.DataFrame(env,index=index)
        df = df.iloc[1:,1:]
        
    if len(env)==m+2 and len(env[0])==n+2:
        index=pd.Index(["0"]+[chr(65+i) for i in range(m)]+["0"])
        df = pd.DataFrame(env,index=index)
        df = df.iloc[1:-1,1:-1]   
    
    columns=[str(i) for i in range(1,n+1)]
    #df.fillna("X",inplace=True)
    #print(df)
    print(tabulate(df, headers=columns, tablefmt='fancy_grid'))
    
        

def evaluation_format(answ, pt_green,pt_red):
    pt_blue=0
    if pt_green!=0:
        pt_blue=pt_red-pt_green
        pt_red=0
    return f"{answ}. Totalizzeresti <span style='color:green'>[{pt_green} safe pt]</span>, \
                                    <span style='color:blue'>[{pt_blue} possible pt]</span>, \
                                    <span style='color:red'>[{pt_red} out of reach pt]</span>.<br>"

def check_num_paths_to(mappa, num_paths_to):
    """
    verifica che la matrice num_paths_to sia conforme alla consegna (perdonando solamente eventuale anomalia su cella (1,1) ).
    """
    if len(num_paths_to) != m+1:
        return evaluation_format("No", 0, 10)+f"Le righe della matrice $num\_paths\_to$ devono essere $m+1=${m+1}, non {len(num_paths_to)}."
    if len(num_paths_to[0]) != n+1:
        return evaluation_format("No", 0, 10)+f"Le colonne della matrice $num\_paths\_to$ devono essere $n+1=${n+1}, non {len(num_paths_to[0])}."
        
    for i in range (0,m):
        if num_paths_to[i][0]!=0:
            return evaluation_format("No", 0, 10)+f"Attenzione, i cammini devono partire dalla cella $(1,1)$ e pertanto $num\_paths\_to[${i}$][0] = 0$"
    for j in range (0,n):
        if num_paths_to[0][j]!=0:
            return evaluation_format("No", 0, 10)+f"Attenzione, i cammini devono partire dalla cella $(1,1)$ e pertanto $num\_paths\_to[0][${j}$] = 0$"
    num_paths_to_forgiving = num_paths_to.copy()
    num_paths_to_forgiving[1][1] = 1
    for i in range(m,0,-1):
        for j in range (n,0,-1):
            if i==1 and j==1:
                return  evaluation_format("Si", 10, 10)+"Non riscontro particolari problemi della tua versione della matrice $num\_paths\_to$."
            if mappa[i][j]=="*"  or (num_paths_to_forgiving[i-1][j]==None and num_paths_to_forgiving[i][j-1]==None):
                if num_paths_to_forgiving[i][j] is not None:
                    print(f"exit 1; i={i}, j={j}, mappa[i][j] = {mappa[i][j]}, num_paths_to_forgiving[i][j]={num_paths_to_forgiving[i][j]}")
                    return  evaluation_format("No", 0, 10)+"Ti avviso: riscontro dei problemi nella tua versione della matrice $num\_paths\_to$."
            else:
                if num_paths_to_forgiving[i-1][j]==None:
                    if num_paths_to_forgiving[i][j]!=num_paths_to_forgiving[i][j-1]:
                        print(f"exit 2; i={i}, j={j}, mappa[i][j] = {mappa[i][j]}, num_paths_to_forgiving[i][j]={num_paths_to_forgiving[i][j]}, num_paths_to_forgiving[i-1][j]={num_paths_to_forgiving[i-1][j]}, num_paths_to_forgiving[i][j-1]={num_paths_to_forgiving[i][j-1]}")
                        return  evaluation_format("No", 0, 10)+"Ti avviso: riscontro dei problemi nella tua versione della matrice $num\_paths\_to$."
                elif num_paths_to_forgiving[i][j-1]==None:
                    if num_paths_to_forgiving[i][j]!=num_paths_to_forgiving[i-1][j]:
                        print(f"exit 3; i={i}, j={j}, mappa[i][j] = {mappa[i][j]}, num_paths_to_forgiving[i][j]={num_paths_to_forgiving[i][j]}, num_paths_to_forgiving[i-1][j]={num_paths_to_forgiving[i-1][j]}, num_paths_to_forgiving[i][j-1]={num_paths_to_forgiving[i][j-1]}")
                        return  evaluation_format("No", 0, 10)+"Ti avviso: riscontro dei problemi nella tua versione della matrice $num\_paths\_to$."
                else: 
                    if num_paths_to_forgiving[i][j]!=num_paths_to_forgiving[i-1][j]+num_paths_to_forgiving[i][j-1]:
                        print(f"exit 4; i={i}, j={j}, mappa[i][j] = {mappa[i][j]}, num_paths_to_forgiving[i][j]={num_paths_to_forgiving[i][j]}, num_paths_to_forgiving[i-1][j]={num_paths_to_forgiving[i-1][j]}, num_paths_to_forgiving[i][j-1]={num_paths_to_forgiving[i][j-1]}")
                        return  evaluation_format("No", 0, 10)+"Ti avviso: riscontro dei problemi nella tua versione della matrice $num\_paths\_to$."

def check_num_paths_from(mappa, num_paths_from):
    """
    verifica che la matrice num_paths sia conforme alla consegna (perdonando solamente eventuale anomalia su cella (1,1) ).
    """
    if len(num_paths_from) != m+2:
        return evaluation_format("No", 0, 10)+f"Le righe della matrice $num\_paths\_from$ devono essere $m+2=${m+2}, non {len(num_paths_from)}."
    if len(num_paths_from[0]) != n+2:
        return evaluation_format("No", 0, 10)+f"Le colonne della matrice $num\_paths\_from$ devono essere $n+2=${n+2}, non {len(num_paths_from[0])}."
        
    for i in range (0,m+1):
        if num_paths_from[i][n+1]!=0:
            return evaluation_format("No", 0, 10)+f"Attenzione, i cammini devono partire dalla cella $(8,9)$ e pertanto $num\_paths\_from[${i}$][10] = 0$"
    for j in range (0,n+1):
        if num_paths_from[m+1][j]!=0:
            return evaluation_format("No", 0, 10)+f"Attenzione, i cammini devono partire dalla cella $(8,9)$ e pertanto $num\_paths\_from[9][${j}$] = 0$"
    num_paths_from_forgiving = num_paths_from.copy()
    num_paths_from_forgiving[m][n] = 1
    for i in range(1,m-1):
        for j in range (1,n-1):
            if mappa[i][j]!="*":
                if num_paths_from_forgiving[i][j]!=num_paths_from_forgiving[i+1][j]+num_paths_from_forgiving[i][j+1]:
                    print(f"exit 1; i={i}, j={j}, mappa[i][j] = {mappa[i][j]}, num_paths_from_forgiving[i][j]={num_paths_from_forgiving[i][j]}, num_paths_from_forgiving[i+1][j]={num_paths_from_forgiving[i+1][j]}, num_paths_from_forgiving[i][j+1]={num_paths_from_forgiving[i][j+1]}")
                    return  evaluation_format("No", 0, 10)+"Ti avviso: riscontro dei problemi nella tua versione della matrice $num\_paths\_from$."
            elif num_paths_from_forgiving[i][j]!=0:
                print(f"exit 2; i={i}, j={j}, mappa[i][j] = {mappa[i][j]}, num_paths_from_forgiving[i][j]={num_paths_from_forgiving[i][j]}")
                return  evaluation_format("No", 0, 10)+"Ti avviso: riscontro dei problemi nella tua versione della matrice $num\_paths\_from$."
    for i in range (1, m):
        if mappa[i][n]!="*":
            if num_paths_from_forgiving[i][n]!=num_paths_from_forgiving[i+1][n]+num_paths_from_forgiving[i][n+1]:    
                print(f"exit 3; i={i}, j={j}, mappa[i][j] = {mappa[i][j]}, num_paths_from_forgiving[i][n]={num_paths_from_forgiving[i][n]}, num_paths_from_forgiving[i+1][n]={num_paths_from_forgiving[i+1][n]}, num_paths_from_forgiving[i][n+1]={num_paths_from_forgiving[i][n+1]}")
                return  evaluation_format("No", 0, 10)+"Ti avviso: riscontro dei problemi nella tua versione della matrice $num\_paths\_from$." 
        elif num_paths_from_forgiving[i][n]!=0:
            print(f"exit 4; i={i}, j={j}, mappa[i][j] = {mappa[i][j]}, num_paths_from_forgiving[i][j]={num_paths_from_forgiving[i][j]}")
            return  evaluation_format("No", 0, 10)+"Ti avviso: riscontro dei problemi nella tua versione della matrice $num\_paths\_from$."
    for j in range (1, n):
        if mappa[m][j]!="*":
            if num_paths_from_forgiving[m][j]!=num_paths_from_forgiving[m+1][j]+num_paths_from_forgiving[m][j+1]:
                print(f"exit 5; i={i}, j={j}, mappa[i][j] = {mappa[i][j]}, num_paths_from_forgiving[m][j]={num_paths_from_forgiving[m+1][j]}, num_paths_from_forgiving[m+1][j+1]={num_paths_from_forgiving[i+1][j]}, num_paths_from_forgiving[i][j+1]={num_paths_from_forgiving[i][j+1]}")
                return  evaluation_format("No", 0, 10)+"Ti avviso: riscontro dei problemi nella tua versione della matrice $num\_paths\_from$." 
        elif num_paths_from_forgiving[m][j]!=0:
            print(f"exit 6; i={i}, j={j}, mappa[i][j] = {mappa[i][j]}, num_paths_from_forgiving[i][j]={num_paths_from_forgiving[i][j]}")
            return  evaluation_format("No", 0, 10)+"Ti avviso: riscontro dei problemi nella tua versione della matrice $num\_paths\_from$."
    return  evaluation_format("Si", 10, 10)+"Non riscontro particolari problemi della tua versione della matrice $num\_paths\_from$."

## Esercizio \[60 pts\]
(campo minato) Conteggio di cammini in una griglia rettangolare con celle proibite.

Bimo cammina sulle celle di un campo minato dalla forma di una griglia rettangolare $m\times n$. Le mine sono indicate da un "*" mentre le altre celle (" ") sono tutte transitabili.
Le mosse consentite portano Bimo dalla cella $(i,j)$ alla cella $(i+1,j)$ oppure $(i,j+1)$, sempre ove queste siano transitabili.
Organizzati per calcolare quanti sono i cammini possibili tra due celle date e per rispondere ad altre domande di questo tipo.

<b>Notice:</b> Anche se ne hai quì ogni opportunità, non ti è però richiesto in alcun modo di scrivere del codice per condurre a termine il tuo esercizio. Puoi fare tutto a mano e vogliamo essere chiari che noi non facciamo alcuna differenza tra i punti conquistati in un modo piuttosto che in un altro (noi guardiamo ai risultati e ci piace che voi vi ingegniate a modo vostro per portarli a casa, in tutta libertà). Sei incoraggiato piuttosto a ricercare l'approccio per tè più pratico, sicuro, e conveniente. E magari quello che puoi trovare più piacevole e stimolante quando svlgi l'esercizio da casa, dove ti suggerisco sperimentare, potrebbe anche essere diverso .
Ciò nononostante, per facilitare chi di voi volesse scrivere del codice a proprio supporto, abbiamo aggiunto alla mappa di $m$ righe ed $n$ colonne una riga e colonna iniziale (di indice zero), fatte interamente di mine, perchè non si crei confusione col fatto che gli indici di liste ed array in programmazione partono da zero.

Un robot, inizialmente situato nella cella $A1=(1,1)$, deve portarsi nella cella $H9=(8,9)$.
Le celle che riportano il simbolo '*' contengono una mina od altre trapole mortali, ed il robot deve evitarle. 
I movimenti base possibili sono il passo verso destra (ad esempio il primo passo potrebbe avvenire dalla cella $A1$ alla cella $A2$) ed il passo verso il basso (ad esempio, come unica altra alternativa per il primo passo il robot potrebbe posrtarsi quindi nella cella $B1$).\
Quanti sono i possibili percorsi che può fare il robot per andare dalla cella $A1$ alla cella $H9$?

In [213]:
visualizza_env(mappa)

╒════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╕
│    │ 1   │ 2   │ 3   │ 4   │ 5   │ 6   │ 7   │ 8   │ 9   │
╞════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╡
│ A  │     │     │     │     │     │     │     │     │ *   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ B  │     │     │     │     │ *   │ *   │     │     │     │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ C  │     │ *   │     │ *   │     │     │     │     │     │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ D  │     │     │     │     │     │     │     │ *   │     │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ E  │     │     │ *   │     │ *   │     │     │     │     │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ F  │     │     │     │     │     │     │ *   │     │     │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ G  │     │ *   │     │     │     │     │     │     │     │
├────┼─────┼─────┼─────┼

__Richieste__:

1. __\[10 pts\]__ A mano o tramite un programma componi la matrice $num\_paths\_to$ di dimensione $(m+1)\times(n+1)$ e tale per cui in $num\_paths\_to[i][j]$ sia riposto il numero di cammini dalla cella $A1=(1,1)$ alla generica cella $(i,j)$, per ogni $i = 0,..., m+1$ e $j = 0,..., n+1$.

In [218]:
num_paths_to = [
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
                [0, 1, 1, 1, 1, 1, 1, 1, 1, None], 
                [0, 1, 2, 3, 4, None, None, 1, 2, 2], 
                [0, 1, None, 3, None, None, None, 1, 3, 5], 
                [0, 1, 1, 4, 4, 4, 4, 5, None, 5], 
                [0, 1, 2, None, 4, None, 4, 9, 9, 14], 
                [0, 1, 3, 3, 7, 7, 11, None, 9, 23], 
                [0, 1, None, 3, 10, 17, 28, 28, 37, 60], 
                [0, 1, 1, 4, 14, 31, 59, 87, 124, 184]
            ]
#num_paths_to=num_percorsi(mappa)

In [219]:
display(Markdown(f"La tua versione attuale della matrice $num\_paths\_to$ è la seguente:"))
visualizza_env(num_paths_to)
display(Markdown(f"<b>Validazione della tua matrice $num\_paths\_to$:</b>"))
display(Markdown(check_num_paths_to(mappa,num_paths_to)))

La tua versione attuale della matrice $num\_paths\_to$ è la seguente:

╒════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╕
│    │   1 │   2 │   3 │   4 │   5 │   6 │   7 │   8 │   9 │
╞════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╡
│ A  │   1 │   1 │   1 │   1 │   1 │   1 │   1 │   1 │ nan │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ B  │   1 │   2 │   3 │   4 │ nan │ nan │   1 │   2 │   2 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ C  │   1 │ nan │   3 │ nan │ nan │ nan │   1 │   3 │   5 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ D  │   1 │   1 │   4 │   4 │   4 │   4 │   5 │ nan │   9 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ E  │   1 │   2 │ nan │   4 │ nan │   4 │   9 │   9 │  14 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ F  │   1 │   3 │   3 │   7 │   7 │  11 │ nan │   9 │  23 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ G  │   1 │ nan │   3 │  10 │  17 │  28 │  28 │  37 │  60 │
├────┼─────┼─────┼─────┼

<b>Validazione della tua matrice $num\_paths\_to$:</b>

exit 4; i=5, j=9, mappa[i][j] =  , num_paths_to_forgiving[i][j]=14, num_paths_to_forgiving[i-1][j]=9, num_paths_to_forgiving[i][j-1]=9


No. Totalizzeresti <span style='color:green'>[0 safe pt]</span>,                                     <span style='color:blue'>[0 possible pt]</span>,                                     <span style='color:red'>[10 out of reach pt]</span>.<br>Ti avviso: riscontro dei problemi nella tua versione della matrice $num\_paths\_to$.

2. __\[10 pts\]__ Componi ora una matrice $num\_paths\_from$ di dimensione $(m+2)\times(n+2)$ e tale per cui in $num\_paths\_from[i][j]$, per ogni $i = 1,..., m+1$ e $j = 1,..., n+1$, sia riposto il numero di cammini dalla generica cella $(i,j)$ alla cella $H9=(8,9)$.

In [224]:
num_paths_from=num_percorsi_reverse(mappa)
print(num_paths_from)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 184, 63, 36, 9, 9, 9, 9, 2, None, 0], [0, 121, 27, 27, None, None, None, 7, 2, 1, 0], [0, 94, None, 27, None, 29, 17, 5, 1, 1, 0], [0, 94, 49, 27, 27, 12, 12, 4, None, 1, 0], [0, 45, 22, None, 15, None, 8, 4, 4, 1, 0], [0, 23, 22, 22, 15, 9, 4, None, 3, 1, 0], [0, 1, None, 7, 6, 5, 4, 3, 2, 1, 0], [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [225]:
display(Markdown(f"La tua versione attuale della matrice $num\_paths\_from$ è la seguente:"))
visualizza_env(num_paths_from)
display(Markdown(f"<b>Validazione della tua matrice $num\_paths\_from$:</b>"))
display(Markdown(check_num_paths_from(mappa, num_paths_from)))

La tua versione attuale della matrice $num\_paths\_from$ è la seguente:

╒════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╕
│    │   1 │   2 │   3 │   4 │   5 │   6 │   7 │   8 │   9 │
╞════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╡
│ A  │ 184 │  63 │  36 │   9 │   9 │   9 │   9 │   2 │ nan │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ B  │ 121 │  27 │  27 │ nan │ nan │ nan │   7 │   2 │   1 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ C  │  94 │ nan │  27 │ nan │  29 │  17 │   5 │   1 │   1 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ D  │  94 │  49 │  27 │  27 │  12 │  12 │   4 │ nan │   1 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ E  │  45 │  22 │ nan │  15 │ nan │   8 │   4 │   4 │   1 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ F  │  23 │  22 │  22 │  15 │   9 │   4 │ nan │   3 │   1 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ G  │   1 │ nan │   7 │   6 │   5 │   4 │   3 │   2 │   1 │
├────┼─────┼─────┼─────┼

<b>Validazione della tua matrice $num\_paths\_from$:</b>

TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'

1. __\[10 pts\]__ Quanti sono i percorsi con partenza in A–1=(1,1) ed arrivo in H–9=(8,9)?

In [10]:
#184
assert(num_paths_to[8][9] == num_paths_from[1][1])
print(num_paths_to[8][9])


184


2. __\[10 pts\]__ Quanti sono i percorsi con partenza in B–3=(2,3) ed arrivo in H–9=(8,9)?

27

3. __\[10 pts\]__ Quanti sono i percorsi con partenza in A–1=(1,1) ed arrivo in F–6=(6,6)?

11

4. __\[10 pts\]__ Quanti sono i percorsi che partono da A–1=(1,1), passano da D-5=(4,5), ed arrivano in H–9=(8,9)?

In [11]:
#48
print(num_paths_to[4][5]*num_paths_from[4][5])

48
